# Science des données dans le cloud : La méthode "Azure ML SDK"

## Introduction

Dans ce notebook, nous allons apprendre à utiliser le SDK Azure ML pour entraîner, déployer et consommer un modèle via Azure ML.

Prérequis :
1. Vous avez créé un espace de travail Azure ML.
2. Vous avez chargé le [jeu de données sur l'insuffisance cardiaque](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) dans Azure ML.
3. Vous avez téléchargé ce notebook dans Azure ML Studio.

Les étapes suivantes sont :

1. Créer une expérience dans un espace de travail existant.
2. Créer un cluster de calcul.
3. Charger le jeu de données.
4. Configurer AutoML en utilisant AutoMLConfig.
5. Exécuter l'expérience AutoML.
6. Explorer les résultats et obtenir le meilleur modèle.
7. Enregistrer le meilleur modèle.
8. Déployer le meilleur modèle.
9. Consommer le point de terminaison.

## Importations spécifiques au SDK Azure Machine Learning


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Initialiser l'espace de travail
Initialisez un objet espace de travail à partir de la configuration enregistrée. Assurez-vous que le fichier de configuration est présent à .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Créer une expérience Azure ML

Créons une expérience nommée 'aml-experiment' dans l'espace de travail que nous venons d'initialiser.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Créer un Cluster de Calcul
Vous devrez créer une [cible de calcul](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) pour votre exécution AutoML.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Données
Assurez-vous d'avoir téléchargé le jeu de données sur Azure ML et que la clé porte le même nom que le jeu de données.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Configuration AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Exécution AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Déployer le meilleur modèle

Exécutez le code suivant pour déployer le meilleur modèle. Vous pouvez consulter l'état du déploiement dans le portail Azure ML. Cette étape peut prendre quelques minutes.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Utiliser le point de terminaison
Vous pouvez ajouter des entrées à l'exemple d'entrée suivant.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
